In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
import pathlib

In [8]:

# Load the data
train_dir = pathlib.Path("../Birds/train/train")
validation_dir = pathlib.Path("../Birds/valid/valid")
test_dir = pathlib.Path("../Birds/test/test")

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

validation_generator = ImageDataGenerator(rescale=1./255)

validation_generator = validation_generator.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

test_generator = ImageDataGenerator(rescale=1./255)

test_generator = test_generator.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

# Load the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

# Add a new top layer for classification
top_model = Sequential([
    Flatten(),
    Dense(256, activation='relu'),
    Dense(3, activation='softmax')
])

# Freeze the pre-trained weights
for layer in base_model.layers:
    layer.trainable = False

# Combine the two models
model = Sequential([
    base_model,
    top_model
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

# Explore the model performance
confusion_matrix = model.evaluate(test_generator)
print('Confusion matrix:')
print(confusion_matrix)

precision, recall, f1_score = model.evaluate(test_generator)
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1_score)

# Plot the precision-recall curve and AUC
precision_recall_curve = model.evaluate(test_generator)
print('AUC:', precision_recall_curve)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '..\\Birds\\train\\train'